In [ ]:
import tensorflow

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
############### This model is saturated ############# go on to the predictions

In [ ]:
tokenizer = Tokenizer()

data = open("coldplay_lyrics.txt", encoding="utf8").read()

In [ ]:
dataset = data.lower().split("\n")
for data in dataset:
    if data=='':
        dataset.remove('')
print(dataset)

In [ ]:
tokenizer.fit_on_texts(dataset)
total_words = len(tokenizer.word_index) + 1

In [ ]:
print(tokenizer.word_index)
print(total_words)

In [ ]:
input_sequences = []
for line in dataset:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequences = token_list[:i+1]
        input_sequences.append(n_gram_sequences)

In [ ]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
for s in input_sequences:
    print(s)

In [ ]:
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tensorflow.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
model = tensorflow.keras.Sequential([
    tensorflow.keras.layers.Embedding(total_words, 32, input_length=max_sequence_len-1),
    tensorflow.keras.layers.Bidirectional(tensorflow.keras.layers.LSTM(128, return_sequences=True)),
    tensorflow.keras.layers.Bidirectional(tensorflow.keras.layers.LSTM(64, return_sequences=True)),
    tensorflow.keras.layers.SimpleRNN(32),
    tensorflow.keras.layers.Dense(256),
    tensorflow.keras.layers.Dense(total_words, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=tensorflow.optimizers.Adam(lr = 0.001), metrics=['accuracy'])

TF_FORCE_GPU_ALLOW_GROWTH=True

In [ ]:
history = model.fit(xs, ys, epochs=15, verbose=1)

In [ ]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.show

In [ ]:
plot_graphs(history, 'accuracy')

In [ ]:
plot_graphs(history, 'loss')

In [ ]:
# model1 = tensorflow.keras.models.load_model('coldplay_lite_model_v1.h5')

In [ ]:
seed_text = "i am bleeding"
next_words = 200  ##More words is more gibberish
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

In [ ]:
model.save('coldplay_lite_model_v1.h5')